In [10]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeCV
import numpy as np
import os
import sys
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import metrics
import itertools
#import other python scripts for further anlaysis
import reshape
import plotFW
import results
# Initialization of directory information:
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
taskFC=reshape.matFiles(dataDir+'motor/MSC03_parcel_corrmat.mat')
restFC=reshape.matFiles(dataDir+'rest/MSC03_parcel_corrmat.mat')

taskSize=taskFC.shape[0]
restSize=restFC.shape[0]
t = np.ones(taskSize, dtype = int)
r=np.zeros(restSize, dtype=int)
#x_train, y_train=reshape.concateFC(taskFC, restFC)
#clf=RidgeClassifier()
clf=LinearSVC()

test_taskFC=reshape.matFiles(dataDir+'motor/MSC04_parcel_corrmat.mat')
test_restFC=reshape.matFiles(dataDir+'rest/MSC04_parcel_corrmat.mat')

In [19]:
def CV_folds(clf, analysis, taskFC, restFC, test_taskFC, test_restFC):
    loo = LeaveOneOut()
    taskSize=taskFC.shape[0]
    restSize=restFC.shape[0]
    t = np.ones(taskSize, dtype = int)
    r=np.zeros(restSize, dtype=int)
    if analysis=='SS':
        df=pd.DataFrame()
        acc_score=[]
        for train_index, test_index in loo.split(taskFC):
            Xtrain_rest, Xtest_rest=restFC[train_index], restFC[test_index]
            Xtrain_task=taskFC[train_index]
            ytrain_rest=r[train_index]
            ytrain_task=t[train_index]
            X_tr=np.concatenate((Xtrain_task, Xtrain_rest))
            y_tr = np.concatenate((ytrain_task,ytrain_rest))
            clf.fit(X_tr,y_tr)
            tmpdf=pd.DataFrame()
            acc_scores_per_fold=[]
            for t_index, te_index in loo.split(test_taskFC):
                Xtest_task=test_taskFC[te_index]
                X_Test = np.concatenate((Xtest_task, Xtest_rest))
                y_Test = np.array([1, 0])
                #test set
                clf.predict(X_Test)
                #Get accuracy of model
                ACCscores=clf.score(X_Test,y_Test)
                acc_scores_per_fold.append(ACCscores)
            tmpdf['inner_fold']=acc_scores_per_fold
            score=tmpdf['inner_fold'].mean()
            acc_score.append(score)
        df['outer_fold']=acc_score
        total_score=df['outer_fold'].mean()
    else:
        df=pd.DataFrame()
        acc_score=[]
        #fold each training set
        for train_index, test_index in loo.split(taskFC):
            Xtrain_rest=restFC[train_index]
            Xtrain_task=taskFC[train_index]
            ytrain_rest=r[train_index]
            ytrain_task=t[train_index]
            X_tr=np.concatenate((Xtrain_task, Xtrain_rest))
            y_tr = np.concatenate((ytrain_task,ytrain_rest))
            clf.fit(X_tr,y_tr)
            tmpdf=pd.DataFrame()
            acc_scores_per_fold=[]
            #fold each testing set
            for t_index, te_index in loo.split(test_taskFC):
                Xtest_rest=test_restFC[te_index]
                Xtest_task=test_taskFC[te_index]
                X_te=np.concatenate((Xtest_task, Xtest_rest))
                y_te=np.array([1, 0])
                #test set
                clf.predict(X_te)
                #Get accuracy of model
                ACCscores=clf.score(X_te,y_te)
                acc_scores_per_fold.append(ACCscores)
            tmpdf['inner_fold']=acc_scores_per_fold
            score=tmpdf['inner_fold'].mean()
            acc_score.append(score)
        df['outer_fold']=acc_score
        total_score=df['outer_fold'].mean()

    #return total_score
    return acc_score

In [20]:
df=CV_folds(clf,'DS', taskFC, restFC, test_taskFC, restFC)

In [11]:
test_taskFC.shape

(10, 55278)

In [12]:
np.random.choice(test_taskFC, 3, replace=False)

ValueError: a must be 1-dimensional

In [18]:
np.random.choice(k, 100, replace=False)

array([ 1.09763036, -0.01899441,  0.01650812, -0.25375739,  0.07873763,
        0.09911291, -0.05544783, -0.41041462, -0.17094646,  0.01864297,
        0.11709865,  0.14827267, -0.1182336 ,  0.42108816, -0.19530186,
        0.13808115,  0.41946645,  0.17813812, -0.08705092, -0.04696185,
        0.14127051, -0.31013964, -0.38706382, -0.27757083,  0.18355256,
       -0.07263059,  0.02838136, -0.25368466,  0.28478869, -0.06630116,
       -0.26838603,  0.29837717,  0.42838237,  0.30933027, -0.39342159,
        0.32971424,  0.32961862, -0.04720519,  0.38405336, -0.05307149,
       -0.38174899,  0.19352586, -0.21244758,  0.03245149,  0.04668725,
        0.05758754, -0.14526185, -0.01285864,  0.21475169, -0.03652233,
       -0.29098356, -0.03300232,  0.09595936,  0.05097642, -0.05901089,
        0.09569546, -0.43641805,  0.11827181, -0.24769562,  0.08081717,
        0.00196297,  0.16288282, -0.08205677,  0.0100271 ,  0.00338862,
        0.09799865, -0.00223716,  0.08169544, -0.32873312, -0.00

In [20]:
foo=np.random.choice(test_taskFC[1], 100, replace=False)
bar=np.random.choice(test_taskFC[2], 100, replace=False)

In [24]:
dsNet=np.empty((10, 100))
dsNet[0]=bar
dsNet[1]=foo

In [28]:
dsNet.shape

(10, 100)

In [27]:
foo.shape

(100,)

In [46]:
df=pd.DataFrame()
nsess=10
features=[100,500,1000,1500,2000,2500,3000,4000,5000,10000,15000,20000,30000,40000,50000]
for feat in features:
    featList=np.empty((10,feat))
    for i in range(nsess):
        f=np.random.choice(test_taskFC[i], feat, replace=False)
        featList[i]=f
    

In [38]:
a=[1,2,3]
b=[4,5,6]
c=[7,8,9]
d={100 :a}
hello=pd.DataFrame(d)

In [47]:
featList.shape

(10, 50000)